In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random

import emphases

/home/mrm5248/conda/envs/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Get stems to annotate
speakers = [str(s) for s in emphases.data.download.LIBRITTS_SPEAKERS]
stems = sorted([
    file.stem for file in (emphases.CACHE_DIR / 'libritts' / 'audio').glob('*.wav')
    if file.stem.split('_')[0] in speakers])
random.seed(emphases.RANDOM_SEED)
random.shuffle(stems)

In [4]:
eighth = len(stems) // 8

# Initialize annotation progress monitor
monitor = {}
for stem in stems[:eighth]:
    monitor[stem] = (0, 8)
for stem in stems[eighth:2 * eighth]:
    monitor[stem] = (0, 4)
for stem in stems[2 * eighth:4 * eighth]:
    monitor[stem] = (0, 2)
for stem in stems[4 * eighth:]:
    monitor[stem] = (0, 1)

In [5]:
# TODO - update monitor based on current progress

In [6]:
with open('monitor-annotation.json', 'w') as file:
    json.dump(monitor, file)